In [1]:
#@title #**1. Install The Required Softwares**

#@markdown Execute this cell first to download and install third-party softwares

from google.colab import files
from IPython.utils import io
from IPython.display import clear_output
import os, re, shutil
import numpy as np, pandas as pd
import tqdm.notebook, subprocess

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

try:
  with tqdm.notebook.tqdm(total = 100, bar_format = TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      %shell pip install rmsd
      %shell pip install ipywidgets
      from ipywidgets import widgets
      pbar.update(10)

      %shell pip install rdkit
      from rdkit import Chem
      import rdkit.Contrib.CalcLigRMSD.CalcLigRMSD as CLR
      pbar.update(10)

      %shell wget https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz
      %shell tar -xzvf autodock_vina_1_1_2_linux_x86.tgz
      %shell cp /content/autodock_vina_1_1_2_linux_x86/bin/vina /usr/bin/vina
      pbar.update(40)

      %shell wget https://ccsb.scripps.edu/adfr/download/1038/ -O adfr.tar.gz
      %shell tar -xf adfr.tar.gz
      %shell mkdir adfr
      %cd /content/ADFRsuite_x86_64Linux_1.0
      %shell yes | ./install.sh -d /content/adfr
      %cd /content
      pbar.update(40)
except subprocess.CalledProcessError:
  print(captured)
  raise

  0%|          | 0/100 [elapsed: 00:00 remaining: ?]

In [4]:
#@title #**2. Upload Receptor and Ligand File**

#@markdown Please upload the **receptor** and **ligand** in the **PDB** format and simultaneously, then select what file being a receptor or a ligand

#@markdown **Note:** The file name should not contain any space

working_dir = os.path.join('/content', 'working')
if (not os.path.isdir(working_dir)):
  os.mkdir(working_dir)
else:
  shutil.rmtree(working_dir)
  os.mkdir(working_dir)
os.chdir(working_dir)

receptor_name = ''
ligand_name = ''

uploaded_files = list(files.upload().keys())
clear_output()

receptor_input = widgets.Dropdown(
    options = uploaded_files,
    description = 'Receptor:',
    layout = widgets.Layout(margin = '0 0 20px 0')
)
display(receptor_input)

ligand_input = widgets.Dropdown(
    options = uploaded_files,
    description = 'Ligand:'
)
display(ligand_input)

button = widgets.Button(
    description = 'Select',
    layout = widgets.Layout(margin = '20px 0 20px 90px')
)
display(button)

receptor_label = widgets.Label(value = 'Receptor:')
display(receptor_label)
ligand_label = widgets.Label(value = 'Ligand:')
display(ligand_label)

def incorporate_names(b):
  global receptor_name, ligand_name
  receptor_name = receptor_input.value
  ligand_name = ligand_input.value
  receptor_label.value = 'Receptor: ' + receptor_name
  ligand_label.value = 'Ligand: ' + ligand_name

button.on_click(incorporate_names)

Dropdown(description='Receptor:', layout=Layout(margin='0 0 20px 0'), options=('protein.pdb', '9Vjy0K.pdb'), v…

Dropdown(description='Ligand:', options=('protein.pdb', '9Vjy0K.pdb'), value='protein.pdb')

Button(description='Select', layout=Layout(margin='20px 0 20px 90px'), style=ButtonStyle())

Label(value='Receptor:')

Label(value='Ligand:')

In [10]:
!rm -rf /content/working/

In [5]:
#@title #**3. Convert Receptor and Ligand File to PDBQT Format**

#@markdown If the receptor and ligand have not been added by hydrogen atoms yet, enable **add_hydrogens** check below

os.chdir(working_dir)

for f in os.listdir():
  if((f != receptor_name) and (f != ligand_name) and (not os.path.isdir(f))):
    os.remove(f)

receptor_pdbqt = re.split('\.', receptor_name)[0] + '.pdbqt'
ligand_pdbqt = re.split('\.', ligand_name)[0] + '.pdbqt'

add_hydrogens = True #@param {type:"boolean"}

if(add_hydrogens):
  receptor_converted = re.split('\.', receptor_name)[0] + '_converted.pdbqt'
  print(f'Convert {receptor_name} to PDBQT format with addition of hydrogen atoms')
  res_receptor = os.system(f'/content/adfr/bin/prepare_receptor -r {receptor_name} -o {receptor_converted} -A hydrogens')

  with open(receptor_converted, 'r') as rc:
    rp = open(receptor_pdbqt, 'w')
    for line in rc:
      if(('ATOM' in line) and (' ' != line[20])):
        line_fix = ''.join([ltr for idx, ltr in enumerate(line) if idx != 12]) #Fix 'atom name' column
        tmp = list(line_fix)
        tmp[15], tmp[16] = tmp[16], tmp[15]
        line_fix = ''.join(tmp)
        rp.write(line_fix)
        continue
      rp.write(line)
    rp.close()

  ligand_H = re.split('\.', ligand_name)[0] + '_h.pdb'
  os.system(f'/content/adfr/bin/obabel {ligand_name} -O {ligand_H} -p')

  ligand_H_fix = re.split('\.', ligand_H)[0] + '_fix.pdb'
  with open(ligand_H, 'r') as lh:
    lhf = open(ligand_H_fix, 'w')
    for line in lh:
      if('ATOM' in line):
        line_fix = line.replace('ATOM  ', 'HETATM')
        lhf.write(line_fix)
        continue
      lhf.write(line)
    lhf.close()


  ligand_H_fix_converted = re.split('\.', ligand_H_fix)[0] + '.pdbqt'
  print(f'Convert {ligand_name} to PDBQT format with addition of hydrogen atoms')
  res_ligand = os.system(f'/content/adfr/bin/prepare_ligand -l {ligand_H_fix} -o {ligand_H_fix_converted} -A hydrogens')

  with open(ligand_H_fix_converted, 'r') as lhfc:
    lp = open(ligand_pdbqt, 'w')
    for line in lhfc:
      if('ATOM' in line):
        continue
      lp.write(line)
    lp.close()
else:
  print(f'Convert {receptor_name} to PDBQT format')
  res_receptor = os.system(f'/content/adfr/bin/prepare_receptor -r {receptor_name}')

  print(f'Convert {ligand_name} to PDBQT format')
  res_ligand = os.system(f'/content/adfr/bin/prepare_ligand -l {ligand_name}')

Convert protein.pdb to PDBQT format with addition of hydrogen atoms
Convert 9Vjy0K.pdb to PDBQT format with addition of hydrogen atoms


In [ ]:
#@title #**4. Find Ligand Center Point**

#@markdown Center point of x, y, z dimension will be used in the docking process.
#@markdown Once this cell has been executed, the center point of the ligand will be displayed as center_x, center_y, and center_z

os.chdir(working_dir)

ligand_coor = []
with open(ligand_name, 'r') as rf:
  for line in rf:
    if (('HETATM' in line)):
      x_coor = float(line[31:38].strip())
      y_coor = float(line[39:46].strip())
      z_coor = float(line[47:54].strip())
      ligand_coor.append([x_coor, y_coor, z_coor])

center_x, center_y, center_z = np.mean(np.array(ligand_coor), axis = 0)

print('center_x:', center_x)
print('center_y:', center_y)
print('center_z:', center_z)

center_x: 10.888549999999999
center_y: 13.7482
center_z: 17.2609


In [ ]:
#@title #**5. Specify Gridbox Dimension**

#@markdown Specify the minimum and maximum size of the gridbox, and the incremental of gridbox size

gridbox_min = 7 #@param {type:'integer'}
gridbox_max = 50 #@param {type:'integer'}
gridbox_inc = 20 #@param {type:'integer'}

In [ ]:
#@title #**6. Run Gridbox Validation**

#@markdown Run this cell to start running the gridbox validation.
#@markdown The output files (pdbqt and log) would be included in downloaded archive after RMSD calculation

output_dir = os.path.join('/content/working', 'output')
if(not os.path.isdir(output_dir)):
  os.mkdir(output_dir)
else:
  shutil.rmtree(output_dir)
  os.mkdir(output_dir)

gridbox_tot = len(list(range(gridbox_min, gridbox_max + 1,gridbox_inc)))

try:
  with tqdm.notebook.tqdm(total = gridbox_tot, bar_format = TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      for size in range(gridbox_min, gridbox_max + 1, gridbox_inc):
        os.system(
            f'vina --receptor {receptor_pdbqt} \
            --ligand {ligand_pdbqt} \
            --center_x {center_x} \
            --center_y {center_y} \
            --center_z {center_z} \
            --size_x {size} \
            --size_y {size} \
            --size_z {size} \
            --out {output_dir}/gridbox_x{size}_y{size}_z{size}.pdbqt \
            --log {output_dir}/gridbox_x{size}_y{size}_z{size}.log'
        )
        pbar.update(1)
except subprocess.CalledProcessError:
  print(captured)
  raise

  0%|          | 0/3 [elapsed: 00:00 remaining: ?]

In [ ]:
#@title #**7. Calculate RMSD**

#@markdown After running this cell, there would be zip-formatted archive contains
#@markdown files of pdbqt, log, rmsd, affinity summary, and gridbox dimension details
#@markdown that automatically will be downloaded

data_dir = os.path.join(output_dir, 'data')
if(not os.path.isdir(data_dir)):
  os.mkdir(data_dir)
else:
  shutil.rmtree(data_dir)
  os.mkdir(data_dir)

if(os.path.isfile(os.path.join(working_dir, 'output.zip'))):
  os.remove(os.path.join(working_dir, 'output.zip'))

# crystal_ligand = Chem.rdmolfiles.MolFromPDBFile(os.path.join(working_dir, ligand_name))
crystal_ligand = os.path.join(working_dir, 'crystal_ligand.pdb')
with io.capture_output() as captured:
  os.system(f'/content/adfr/bin/obabel {ligand_pdbqt} -O {crystal_ligand}')

gridbox_rmsd = []
for pdbqt_output in os.listdir(output_dir):
    if(('.pdbqt' in pdbqt_output) and (os.path.getsize(os.path.join(output_dir, pdbqt_output)) > 0)):
      pdbqt_output_file = os.path.join(output_dir, pdbqt_output)
    pdbqt_output_file_first_model = os.path.join(output_dir, re.split('\.', pdbqt_output)[0] + '_first_model.pdbqt')
    with open(pdbqt_output_file, 'r') as pof:
      poffm = open(pdbqt_output_file_first_model, 'w')
      for line in pof:
        if('ENDMDL' in line):
          break
        poffm.write(line)
      poffm.close()

    first_model = os.path.join(output_dir, re.split('\.', pdbqt_output_file_first_model)[0] + '.pdb')
    with io.capture_output() as captured:
      os.system(f'/content/adfr/bin/obabel {pdbqt_output_file_first_model} -O {first_model}')

    # docked_ligand = Chem.rdmolfiles.MolFromPDBFile(os.path.join(output_dir, first_model))
    # ligand_rmsd = CLR.CalcLigRMSD(docked_ligand, crystal_ligand, rename_lig2 = False)

    docked_ligand = os.path.join(output_dir, first_model)
    processResult = subprocess.run(['calculate_rmsd', docked_ligand, crystal_ligand, '--reorder'], capture_output=True)

    # ligand_rmsd = float(processResult.stdout.strip())
    # except Exception as e:
    # print(f"Error during RMSD calculation: {str(e)}")
    #  print(f"stderr: {processResult.stderr}")


    ligand_rmsd = float(processResult.stdout)
    affinity = 0
    with open(pdbqt_output_file, 'r') as f:
      for line in f:
        if('VINA RESULT' in line):
          affinity = float(line[24:30].strip())
          break

    gridbox_rmsd.append([re.split('\.', pdbqt_output)[0], ligand_rmsd, affinity])

    os.remove(pdbqt_output_file_first_model)
    os.remove(first_model)

rmsd_df = pd.DataFrame(gridbox_rmsd, columns = ['Gridbox', 'RMSD', 'Affinity']).sort_values(by = ['RMSD'])
rmsd_df.to_csv(os.path.join(data_dir, 'rmsd.csv'), index = False)

gridbox_cp = [['center_x', center_x], ['center_y', center_y], ['center_z', center_z]]
gridbox_cp_df = pd.DataFrame(gridbox_cp, columns = ['Center Point', 'Coordinate'])
gridbox_cp_df.to_csv(os.path.join(data_dir, 'gridbox_center_point.csv'), index = False)

shutil.copyfile(os.path.join(working_dir, receptor_pdbqt), os.path.join(data_dir, receptor_pdbqt))
shutil.copyfile(os.path.join(working_dir, ligand_pdbqt), os.path.join(data_dir, ligand_pdbqt))

!zip -r /content/working/output.zip ./output/*
files.download(os.path.join(working_dir, 'output.zip'))

NameError: name 'pdbqt_output_file' is not defined

# Alternative Code if Error Above

In [ ]:
import subprocess
import os
import pandas as pd
import shutil
import re

data_dir = os.path.join(output_dir, 'data')
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)
else:
    shutil.rmtree(data_dir)
    os.mkdir(data_dir)

if os.path.isfile(os.path.join(working_dir, 'output.zip')):
    os.remove(os.path.join(working_dir, 'output.zip'))

crystal_ligand = os.path.join(working_dir, 'crystal_ligand.pdb')
with io.capture_output() as captured:
    os.system(f'/content/adfr/bin/obabel {ligand_pdbqt} -O {crystal_ligand}')

gridbox_rmsd = []
for pdbqt_output in os.listdir(output_dir):
    if '.pdbqt' in pdbqt_output and os.path.getsize(os.path.join(output_dir, pdbqt_output)) > 0:
        pdbqt_output_file = os.path.join(output_dir, pdbqt_output)
        pdbqt_output_file_first_model = os.path.join(output_dir, re.split('\.', pdbqt_output)[0] + '_first_model.pdbqt')

        # Check if pdbqt_output_file exists
        if not os.path.isfile(pdbqt_output_file):
            print(f"File not found: {pdbqt_output_file}")
            continue  # Skip this iteration if file doesn't exist

        # Create the first model PDBQT
        with open(pdbqt_output_file, 'r') as pof:
            poffm = open(pdbqt_output_file_first_model, 'w')
            for line in pof:
                if 'ENDMDL' in line:
                    break
                poffm.write(line)
            poffm.close()

        # Ensure the first model file was created
        if not os.path.isfile(pdbqt_output_file_first_model):
            print(f"First model file not created: {pdbqt_output_file_first_model}")
            continue

        # Convert first model PDBQT to PDB
        first_model = os.path.join(output_dir, re.split('\.', pdbqt_output_file_first_model)[0] + '.pdb')
        with io.capture_output() as captured:
            os.system(f'/content/adfr/bin/obabel {pdbqt_output_file_first_model} -O {first_model}')

        # Check if the first model PDB was created
        if not os.path.isfile(first_model):
            print(f"First model PDB file not created: {first_model}")
            continue

        docked_ligand = os.path.join(output_dir, first_model)

        # Run RMSD calculation and capture both stdout and stderr
        processResult = subprocess.run(
            ['calculate_rmsd', docked_ligand, crystal_ligand, '--reorder'],
            capture_output=True,
            text=True  # Ensure output is captured as string (not bytes)
        )

        # Check if process ran successfully and capture RMSD
        if processResult.returncode != 0:
            # Check for the error message format and extract RMSD value
            match = re.search(r"Error during RMSD calculation: ([\d.]+)", processResult.stderr)
            if match:
                ligand_rmsd = float(match.group(1))  # Extract the numeric value
            else:
                print(f"Unexpected error during RMSD calculation: {processResult.stderr}")
                ligand_rmsd = None
        elif not processResult.stdout.strip():
            print(f"Empty RMSD output for {docked_ligand}")
            ligand_rmsd = None  # Handle empty output
        else:
            try:
                ligand_rmsd = float(processResult.stdout.strip())
            except ValueError:
                print(f"Could not convert RMSD output to float: {processResult.stdout}")
                ligand_rmsd = None  # Handle conversion error

        # Extract affinity from the PDBQT file
        affinity = 0
        with open(pdbqt_output_file, 'r') as f:
            for line in f:
                if 'VINA RESULT' in line:
                    affinity = float(line[24:30].strip())
                    break

        gridbox_rmsd.append([re.split('\.', pdbqt_output)[0], ligand_rmsd, affinity])

        os.remove(pdbqt_output_file_first_model)
        os.remove(first_model)

# Create DataFrame for RMSD and Affinity values and save as CSV
rmsd_df = pd.DataFrame(gridbox_rmsd, columns=['Gridbox', 'RMSD', 'Affinity']).sort_values(by=['RMSD'])
rmsd_df.to_csv(os.path.join(data_dir, 'rmsd.csv'), index=False)

# Save gridbox center points
gridbox_cp = [['center_x', center_x], ['center_y', center_y], ['center_z', center_z]]
gridbox_cp_df = pd.DataFrame(gridbox_cp, columns=['Center Point', 'Coordinate'])
gridbox_cp_df.to_csv(os.path.join(data_dir, 'gridbox_center_point.csv'), index=False)

# Copy receptor and ligand PDBQT files to data directory
shutil.copyfile(os.path.join(working_dir, receptor_pdbqt), os.path.join(data_dir, receptor_pdbqt))
shutil.copyfile(os.path.join(working_dir, ligand_pdbqt), os.path.join(data_dir, ligand_pdbqt))

# Zip output files and prompt download
!zip -r /content/working/output.zip ./output/*
files.download(os.path.join(working_dir, 'output.zip'))

Unexpected error during RMSD calculation: 0.11236499304321526

Unexpected error during RMSD calculation: 0.14295023480039404

Unexpected error during RMSD calculation: 0.10905494758776597

  adding: output/data/ (stored 0%)
  adding: output/data/rmsd.csv (deflated 30%)
  adding: output/data/3qip_receptor.pdbqt (deflated 73%)
  adding: output/data/gridbox_center_point.csv (deflated 24%)
  adding: output/data/3qip_ligand.pdbqt (deflated 67%)
  adding: output/gridbox_x27_y27_z27.log (deflated 63%)
  adding: output/gridbox_x27_y27_z27.pdbqt (deflated 67%)
  adding: output/gridbox_x47_y47_z47.log (deflated 61%)
  adding: output/gridbox_x47_y47_z47.pdbqt (deflated 82%)
  adding: output/gridbox_x7_y7_z7.log (deflated 63%)
  adding: output/gridbox_x7_y7_z7.pdbqt (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>